## Python Importation

In [3]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import bk.compute
import bk.plot
import bk.load

import scipy.signal
import neuroseries as nts

## Magic Command

In [4]:
%matplotlib qt

## Session Init

In [5]:
path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
os.chdir(path)
bk.load.path = path
bk.load.rat = 8
bk.load.session = 'Rat08-20130713'
bk.load.day = 6

## Loading data

In [6]:
neurons, metadata = bk.load.loadSpikeData(path)
states = bk.load.states()

Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


## Computing spectrograms

In [7]:
start = 15516.0
end = 16405.0

# start = 16120
# end = 16200
interval = nts.IntervalSet(start,end,time_units='s')

In [52]:
lfp = bk.load.lfp(start,end,166,13)
f,t,Sxx = scipy.signal.spectrogram(lfp.as_units('s'),fs=1250,mode= 'psd',nperseg=10000,noverlap=5000)
t = t +lfp.as_units('s').index[0]
spec = Sxx[f<20]
spec_corr = np.corrcoef(spec,rowvar=False)

Load LFP from Rat08-20130713.lfp


## Computing Neurons correlations

In [83]:
hpc_neurons,t_n = bk.compute.binSpikes(neurons[metadata.Region == 'Hpc'],1,start,end)

In [54]:
np.corrcoef(hpc_neurons,rowvar=False)

In [55]:
plt.figure()
s = plt.subplot(4,1,1)
plt.plot(lfp.as_units('s'))
bk.plot.intervals(states['sws'],col='grey')
bk.plot.intervals(states['Rem'])
plt.subplot(4,1,2,sharex=s)
plt.pcolormesh(t, f, Sxx)
plt.ylim(0,100)
plt.clim(0,5000)
plt.subplot(4,1,3,sharex=s)
plt.pcolormesh(t,t,spec_corr)
plt.clim(0,1)
plt.subplot(4,1,4,sharex = s)
plt.pcolormesh(t_n,t_n,hpc_corr)
plt.xlim(start,end)

/home/billel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  import sys
/home/billel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  # This is added back by InteractiveShellApp.init_path()


(15516.0, 16405.0)

In [ ]:
scipy.io.loadmat('')

In [13]:
start = 15516.0
end = 16405.0

# start = 16120
# end = 16200
interval = nts.IntervalSet(start,end,time_units='s')

stru = ['Hpc','BLA','Pir']
channel = [13,101,80]

for n,(s,c) in enumerate(zip(stru,channel),0):
    lfp = bk.load.lfp(start,end,166,c)
    f,t,Sxx = scipy.signal.spectrogram(lfp.as_units('s'),fs=1250,mode= 'psd',nperseg=10000,noverlap=5000)
    t = t +lfp.as_units('s').index[0]
    spec = Sxx[f<100]
    spec_corr = np.corrcoef(spec,rowvar=False)
    
    stru_neurons,t_n = bk.compute.binSpikes(neurons[metadata.Region == s],1,start,end)
    neurons_corr = np.corrcoef(stru_neurons,rowvar=False)
    
    plt.figure()
    sub = plt.subplot(4,1,1)
    plt.title('LFP')
    plt.plot(lfp.as_units('s'))
    bk.plot.intervals(states['sws'],col='grey')
    bk.plot.intervals(states['Rem'])
   
    plt.subplot(4,1,2,sharex=sub)
    plt.title('Spectrogram')
    plt.pcolormesh(t, f, Sxx)
    plt.ylim(0,100)
    plt.clim(0,5000)
    
    
    plt.subplot(4,1,3,sharex=sub)
    plt.title('SpectrogramCorr')
    plt.pcolormesh(t,t,spec_corr)
    plt.clim(0,1)
    
    plt.subplot(4,1,4,sharex = sub)
    plt.title('SpikeTrainCorr') 
    plt.pcolormesh(t_n,t_n,neurons_corr)
    plt.xlim(start,end)
    
    plt.tight_layout()
    plt.suptitle(s)

Load LFP from Rat08-20130713.lfp


/home/billel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
/home/billel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.


Load LFP from Rat08-20130713.lfp
Load LFP from Rat08-20130713.lfp


In [12]:
stru_neurons.shape

(76, 888)

In [93]:
fig, axes = plt.subplots(nrows=3, ncols=2)

In [98]:
for ax in axes.flatten():
    ax.plot(lfp)